In [1]:
# XRD Deep learning CNN
import tensorflow as tf
import numpy as np
import os
#from utils.utilities import *
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

tf.set_random_seed(777)  # reproducibility

# hyper parameters
num_files = 6010
num_files_te = 1000
vld_num_files = 1000
X_length = 4511
X_length_ = 4501

level_num = 5

lstm_layers = 2        # Number of layers

batch_size_ = 1000       # Batch size
batch_size = 1000 

learning_rate = 0.001  # Learning rate (default is 0.001)
epochs = 1

# Fixed
n_classes = 38


In [2]:
#----------------------------------------------------------------------------------------------------------------
#----------------------------------------------------------------------------------------------------------------
graph = tf.Graph()
with graph.as_default():
    
    filename_queue = tf.train.string_input_producer([("D:\\AL_DATA\\Sr_Li_Al_O_Ver_001\\Training\\%d.csv" % i) for i in range(num_files)], 
                                                    shuffle=True, name='filename_queue')

    reader = tf.TextLineReader()
    key, value = reader.read(filename_queue)

    record_defaults = [[0] for _ in range(X_length)]
    record_defaults = [tf.constant([0], dtype=tf.float32) for _ in range(X_length)]

    xy_data = tf.decode_csv(value, record_defaults=record_defaults)
    xy_data = tf.stack(xy_data)
    #xy_data = tf.transpose(xy_data)
    
    A_1=tf.cast(xy_data[-4], tf.float32)
    A_2=tf.cast(xy_data[-3], tf.float32)
    A_3=tf.cast(xy_data[-2], tf.float32)
    
    y_1=tf.cast(xy_data[-7], tf.int32)
    y_2=tf.cast(xy_data[-6], tf.int32)
    y_3=tf.cast(xy_data[-5], tf.int32)
    
    y_1 = tf.cond(tf.greater_equal(A_1, 0.8), lambda:y_1+152, lambda:y_1)
    y_1 = tf.cond(tf.greater_equal(A_1, 0.6) & tf.less(A_1, 0.8), lambda:y_1+114, lambda:y_1)
    y_1 = tf.cond(tf.greater_equal(A_1, 0.4) & tf.less(A_1, 0.6), lambda:y_1+76, lambda:y_1)
    y_1 = tf.cond(tf.greater_equal(A_1, 0.2) & tf.less(A_1, 0.4), lambda:y_1+38, lambda:y_1)
    
    y_2 = tf.cond(tf.greater_equal(A_2, 0.8), lambda:y_2+152, lambda:y_2)
    y_2 = tf.cond(tf.greater_equal(A_2, 0.6) & tf.less(A_2, 0.8), lambda:y_2+114, lambda:y_2)
    y_2 = tf.cond(tf.greater_equal(A_2, 0.4) & tf.less(A_2, 0.6), lambda:y_2+76, lambda:y_2)
    y_2 = tf.cond(tf.greater_equal(A_2, 0.2) & tf.less(A_2, 0.4), lambda:y_2+38, lambda:y_2)
   
    y_3 = tf.cond(tf.greater_equal(A_3, 0.8), lambda:y_3+152, lambda:y_3)
    y_3 = tf.cond(tf.greater_equal(A_3, 0.6) & tf.less(A_3, 0.8), lambda:y_3+114, lambda:y_3)
    y_3 = tf.cond(tf.greater_equal(A_3, 0.4) & tf.less(A_3, 0.6), lambda:y_3+76, lambda:y_3)
    y_3 = tf.cond(tf.greater_equal(A_3, 0.2) & tf.less(A_3, 0.4), lambda:y_3+38, lambda:y_3)
   
    
    
    y_1=tf.one_hot(y_1, n_classes*level_num)
    y_2=tf.one_hot(y_2, n_classes*level_num)
    y_3=tf.one_hot(y_3, n_classes*level_num)

   
    y_data = y_1 + y_2 + y_3  
    
    #y_1=y_1*xy_data[-4]
    #y_2=y_2*xy_data[-3] 
    #y_3=y_3*xy_data[-2]
    
    #y_data_ = tf.stack([y_1, y_2, y_3])  
    #y_data_ = tf.reshape(y_data_, [114])
    
    y_data = tf.to_float(y_data)
    
    print(y_data)
    
    X_train, y_train,  y_train_p, y_train_ind = tf.train.batch([xy_data[:-10], y_data,  xy_data[-4:-1],
                                                                xy_data[-1:]], batch_size = batch_size_)
    y_train_p, _ = tf.nn.top_k(y_train_p, k=3, sorted=True)
    y_train_p = tf.reshape(y_train_p, [batch_size_, 3])
    #X_tr, X_vld = tf.split(X_train, [split_size_tr, split_size_vld], 0)
    #y_tr, y_vld = tf.split(y_train, [split_size_tr, split_size_vld], 0)
    #y_tr_p, y_vld_p = tf.split(y_train_p, [split_size_tr, split_size_vld], 0)
    print(X_train, y_train, y_train_p, y_train_ind)
#==========================================================================VALIDATION SET==========================================


    filename_queue_vld = tf.train.string_input_producer([("D:\\AL_DATA\\Sr_Li_Al_O_Ver_001\\Validation\\%d.csv" % i) 
                                                         for i in range(vld_num_files)], shuffle=True, name='filename_queue')
    
    
    reader_vld = tf.TextLineReader()
    key, value_vld = reader.read(filename_queue)
    
    record_defaults = [[0] for _ in range(X_length)]
    record_defaults = [tf.constant([0], dtype=tf.float32) for _ in range(X_length)]
    
    xy_data_vld = tf.decode_csv(value_vld, record_defaults=record_defaults)
    xy_data_vld = tf.stack(xy_data_vld)
    #xy_data = tf.transpose(xy_data)
    
    A_1_vld=tf.cast(xy_data_vld[-4], tf.float32)
    A_2_vld=tf.cast(xy_data_vld[-3], tf.float32)
    A_3_vld=tf.cast(xy_data_vld[-2], tf.float32)
    
    y_1_vld=tf.cast(xy_data_vld[-7], tf.int32)
    y_2_vld=tf.cast(xy_data_vld[-6], tf.int32)
    y_3_vld=tf.cast(xy_data_vld[-5], tf.int32)
    
    y_1_vld = tf.cond(tf.greater_equal(A_1_vld, 0.8), lambda:y_1_vld+152, lambda:y_1_vld)
    y_1_vld = tf.cond(tf.greater_equal(A_1_vld, 0.6) & tf.less(A_1_vld, 0.8), lambda:y_1_vld+114, lambda:y_1_vld)
    y_1_vld = tf.cond(tf.greater_equal(A_1_vld, 0.4) & tf.less(A_1_vld, 0.6), lambda:y_1_vld+76, lambda:y_1_vld)
    y_1_vld = tf.cond(tf.greater_equal(A_1_vld, 0.2) & tf.less(A_1_vld, 0.4), lambda:y_1_vld+38, lambda:y_1_vld)
    
    y_2_vld = tf.cond(tf.greater_equal(A_2_vld, 0.8), lambda:y_2_vld+152, lambda:y_2_vld)
    y_2_vld = tf.cond(tf.greater_equal(A_2_vld, 0.6) & tf.less(A_2_vld, 0.8), lambda:y_2_vld+114, lambda:y_2_vld)
    y_2_vld = tf.cond(tf.greater_equal(A_2_vld, 0.4) & tf.less(A_2_vld, 0.6), lambda:y_2_vld+76, lambda:y_2_vld)
    y_2_vld = tf.cond(tf.greater_equal(A_2_vld, 0.2) & tf.less(A_2_vld, 0.4), lambda:y_2_vld+38, lambda:y_2_vld)
   
    y_3_vld = tf.cond(tf.greater_equal(A_3_vld, 0.8), lambda:y_3_vld+152, lambda:y_3_vld)
    y_3_vld = tf.cond(tf.greater_equal(A_3_vld, 0.6) & tf.less(A_3_vld, 0.8), lambda:y_3_vld+114, lambda:y_3_vld)
    y_3_vld = tf.cond(tf.greater_equal(A_3_vld, 0.4) & tf.less(A_3_vld, 0.6), lambda:y_3_vld+76, lambda:y_3_vld)
    y_3_vld = tf.cond(tf.greater_equal(A_3_vld, 0.2) & tf.less(A_3_vld, 0.4), lambda:y_3_vld+38, lambda:y_3_vld)
        
     
    y_1_vld=tf.one_hot(y_1_vld, n_classes*level_num)
    y_2_vld=tf.one_hot(y_2_vld, n_classes*level_num)
    y_3_vld=tf.one_hot(y_3_vld, n_classes*level_num)
    

    y_data_vld = y_1_vld + y_2_vld + y_3_vld


    y_data_vld = tf.to_float(y_data_vld)

    print(y_data_vld)
    
    X_vld, y_vld, y_vld_p, y_vld_ind = tf.train.batch([xy_data_vld[:-10], y_data_vld, xy_data_vld[-4:-1], 
                                                       xy_data_vld[-1:]], batch_size = batch_size_)
    y_vld_p, _ = tf.nn.top_k(y_vld_p, k=3, sorted=True)
    y_vld_p = tf.reshape(y_vld_p, [batch_size_, 3])
    #X_tr, X_vld = tf.split(X_train, [split_size_tr, split_size_vld], 0)
    #y_tr, y_vld = tf.split(y_train, [split_size_tr, split_size_vld], 0)
    #y_tr_p, y_vld_p = tf.split(y_train_p, [split_size_tr, split_size_vld], 0)
    print(X_vld, y_vld, y_vld_p, y_vld_ind)

Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensors(tensor).repeat(num_epochs)`.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.TextLin

In [3]:

with graph.as_default():
    
    inputs_ = tf.placeholder(tf.float32, [None, X_length_, 1], name = 'inputs')

    labels_1 = tf.placeholder(tf.float32, [None, n_classes*level_num], name = 'labels_1')

    logit_num = tf.placeholder(tf.int32, [None, 3], name = 'logits_Top_3')
    label_num = tf.placeholder(tf.int32, [None, 3], name = 'labels_Top_3')
    keep_prob_ = tf.placeholder(tf.float32, name = 'keep')
    learning_rate_ = tf.placeholder(tf.float32, name = 'learning_rate')
 

   #-------------------------------------------------------------------------------------------------------------------------- 
       
    conv = tf.layers.conv1d(inputs=inputs_, filters=64, kernel_size=30, strides=1,padding='same', kernel_initializer=tf.contrib.layers.xavier_initializer(), activation = tf.nn.relu)
    max_pool = tf.layers.max_pooling1d(inputs=conv, pool_size=3, strides=2, padding='same')
    print(max_pool.shape)
    
    conv1 = tf.layers.conv1d(inputs=max_pool, filters=64, kernel_size=25, strides=1,padding='same', kernel_initializer=tf.contrib.layers.xavier_initializer(), activation = tf.nn.relu)
    max_pool_1 = tf.layers.max_pooling1d(inputs=conv1, pool_size=3, strides=2, padding='same')
    print(max_pool_1.shape)
      
    conv2 = tf.layers.conv1d(inputs=max_pool_1, filters=64, kernel_size=20, strides=1, padding='same', kernel_initializer=tf.contrib.layers.xavier_initializer(), activation = tf.nn.relu)
    max_pool_2 = tf.layers.max_pooling1d(inputs=conv2, pool_size=2, strides=2, padding='same')
    print(max_pool_2.shape)
    
    conv3 = tf.layers.conv1d(inputs=max_pool_2, filters=64, kernel_size=15, strides=1, padding='same', kernel_initializer=tf.contrib.layers.xavier_initializer(), activation = tf.nn.relu)
    max_pool_3 = tf.layers.max_pooling1d(inputs=conv3, pool_size=1, strides=2, padding='same')
    print(max_pool_3.shape)
    
    conv4 = tf.layers.conv1d(inputs=max_pool_3, filters=64, kernel_size=10, strides=1, padding='same', kernel_initializer=tf.contrib.layers.xavier_initializer(), activation = tf.nn.relu)
    max_pool_4 = tf.layers.max_pooling1d(inputs=conv4, pool_size=1, strides=2, padding='same')
    print(max_pool_4.shape)
    
    #-------------------------------------------------------------------------------------------------------------------------- 
    # Construct the LSTM inputs and LSTM cells
    lstm_in = tf.transpose(max_pool_4, [1,0,2]) # reshape into (seq_len, N, channels)
    
    seq_len=lstm_in.shape[0]
    n_channels=lstm_in.shape[2]
    lstm_size = lstm_in.shape[2]*level_num
    
    lstm_in = tf.reshape(lstm_in, [-1, n_channels]) # Now (seq_len*N, n_channels)
    
    # To cells
    lstm_in = tf.layers.dense(lstm_in, lstm_size, activation=None) # or tf.nn.relu, tf.nn.sigmoid, tf.nn.tanh?
    
    # Open up the tensor into a list of seq_len pieces
    lstm_in = tf.split(lstm_in, seq_len, 0)
    
    # Add LSTM layers
    lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
    drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob_)
    cell = tf.contrib.rnn.MultiRNNCell([drop] * lstm_layers)
    initial_state = cell.zero_state(batch_size, tf.float32)
    
    outputs, final_state = tf.contrib.rnn.static_rnn(cell, lstm_in, dtype=tf.float32,
                                                     initial_state = initial_state)
    
    # We only need the last output tensor to pass into a classifier
    logits_1 = tf.layers.dense(outputs[-1], n_classes*level_num, name='logits')
    
    # Cost function and optimizer
    #cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits_1, labels=labels_))
    cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits_1,labels=labels_1))
    
    # Grad clipping
    train_op = tf.train.AdamOptimizer(learning_rate_)

    gradients = train_op.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients]
    optimizer = train_op.apply_gradients(capped_gradients)
    
    # Accuracy
    #correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(labels_, 1))
    correct_pred = tf.equal(logit_num, label_num)
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name='accuracy')    
    
    #--------------------------------------------------------------------------------------------------------------------------    



For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use keras.layers.conv1d instead.
Instructions for updating:
Use keras.layers.max_pooling1d instead.
(?, 2251, 64)
(?, 1126, 64)
(?, 563, 64)
(?, 282, 64)
(?, 141, 64)
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [4]:

validation_acc = []
validation_loss = []

train_acc = []
train_loss = []


with graph.as_default():
    saver = tf.train.Saver()
with tf.Session(graph=graph) as sess:

    
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, tf.train.latest_checkpoint('D:\\AL_DATA\\Sr_Li_Al_O_Ver_001\\checkpoints-cnn'))
    
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    
    # check data
        
    iteration = 1
    # Loop over epochs
    for e in range(epochs):

        # Loop over batches
        for i in  range(600):    
            state = sess.run(initial_state)


            X_tr, y_tr, y_ind = sess.run([X_train, y_train, y_train_ind])
            #X_tr_a, X_vld_a, y_tr_a, y_vld_a = train_test_split(X_tr, y_tr, test_size=0.2, random_state=42)
            
            
            X_tr= np.reshape(X_tr, (-1, 4501, 1)) 

         
            feed = {inputs_ : X_tr, labels_1 : y_tr,  keep_prob_ : 0.5, initial_state : state, learning_rate_ : learning_rate}
            
            # Loss
            loss, _ , logit, state= sess.run([cost, optimizer, logits_1, final_state], feed_dict = feed)            
            
            
            y_lab = np.empty([batch_size_, 3])
            y_logit = np.empty([batch_size_, 3])
            
            for i in range(batch_size_):
                if y_ind[i,0] == 2:
                    
                    y_lab[i]=np.argsort(y_tr[i])[-3:]
                    y_lab[i]=np.sort(y_lab[i])
                    
                    y_logit[i]=np.argsort(logit[i])[-3:]
                    y_logit[i]=np.sort(y_logit[i])
                    
                elif y_ind[i,0] == 1:
                    
                    z=np.argsort(y_tr[i])[-2:]
                    y_lab[i]=np.append(z, [0])
                    y_lab[i]=np.sort(y_lab[i])
                    
                    z_=np.argsort(logit[i])[-2:]
                    y_logit[i]=np.append(z_, [0])
                    y_logit[i]=np.sort(y_logit[i])
                    
                elif y_ind[i,0] == 0:
                    
                    z=np.argsort(y_tr[i])[-1:]
                    y_lab[i]=np.append(z, [0,0])
                    y_lab[i]=np.sort(y_lab[i])
                    
                    z_=np.argsort(logit[i])[-1:]
                    y_logit[i]=np.append(z_, [0,0])
                    y_logit[i]=np.sort(y_logit[i])
                else:
                    print('Something Wrong happened!!!')       
 
            
            feed = {logit_num : y_logit, label_num: y_lab}

            acc = sess.run(accuracy, feed_dict = feed)
            train_acc.append(acc)
            train_loss.append(loss)                   

            print("Epoch: {}/{}".format(e, epochs),
                    "Iteration: {:d}".format(iteration),
                    "Train loss: {:6f}".format(loss),
                    "Train acc: {:.6f}".format(acc))

###================================================================ VALIDATION =====================================
            if (iteration %5 == 0):
                val_state = sess.run(cell.zero_state(batch_size, tf.float32))
                
                X_vd, y_vd,  y_ind_vd = sess.run([X_vld, y_vld,  y_vld_ind])
                X_vd= np.reshape(X_vd, (-1, 4501, 1))
            

                feed = {inputs_ : X_vd, labels_1 : y_vd,  keep_prob_ : 1.0, initial_state : val_state}
            
                # Loss
                loss_vd,  logit_vd , state_v= sess.run([cost,  logits_1, final_state], feed_dict = feed)            
                
            
                y_lab_vd = np.empty([batch_size_, 3])
                y_logit_vd = np.empty([batch_size_, 3])


                for i in range(batch_size_):
                    if y_ind_vd[i,0] == 2:
                    
                        y_lab_vd[i]=np.argsort(y_vd[i])[-3:]
                        y_lab_vd[i]=np.sort(y_lab_vd[i])
                        
                        y_logit_vd[i]=np.argsort(logit_vd[i])[-3:]
                        y_logit_vd[i]=np.sort(y_logit_vd[i])
                    
                    elif y_ind_vd[i,0] == 1:
                    
                        z=np.argsort(y_vd[i])[-2:]
                        y_lab_vd[i]=np.append(z, [0])
                        y_lab_vd[i]=np.sort(y_lab_vd[i])
                    
                        z_=np.argsort(logit_vd[i])[-2:]
                        y_logit_vd[i]=np.append(z_, [0])
                        y_logit_vd[i]=np.sort(y_logit_vd[i])
                    
                    elif y_ind_vd[i,0] == 0:
                    
                        z=np.argsort(y_vd[i])[-1:]
                        y_lab_vd[i]=np.append(z, [0,0])
                        y_lab_vd[i]=np.sort(y_lab_vd[i])
                    
                        z_=np.argsort(logit_vd[i])[-1:]
                        y_logit_vd[i]=np.append(z_, [0,0])
                        y_logit_vd[i]=np.sort(y_logit_vd[i])
                    else:
                        print('Something Wrong happened!!!')
                feed_1 = {logit_num : y_logit_vd, label_num: y_lab_vd}
            
                # Loss
                acc_vd = sess.run(accuracy, feed_dict = feed_1)
                train_acc.append(acc_vd)
                train_loss.append(loss_vd)            


                print("Epoch: {}/{}".format(e, epochs),
                        "Iteration: {:d}".format(iteration),
                        "Validation loss: {:6f}".format(loss_vd),
                        "Validation acc: {:.6f}".format(acc_vd))
                
                
                # Store
                validation_acc.append(acc_vd)
                validation_loss.append(loss_vd)
            
            iteration += 1 
            
        saver.save(sess,"D:\\AL_DATA\\Sr_Li_Al_O_Ver_001\\checkpoints-cnn\\xrd_class.ckpt")
    coord.request_stop()
    coord.join(threads)
    

Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Epoch: 0/1 Iteration: 1 Train loss: 0.693148 Train acc: 0.040667
Epoch: 0/1 Iteration: 2 Train loss: 0.674672 Train acc: 0.041333
Epoch: 0/1 Iteration: 3 Train loss: 0.583201 Train acc: 0.043667
Epoch: 0/1 Iteration: 4 Train loss: 0.498759 Train acc: 0.040333
Epoch: 0/1 Iteration: 5 Train loss: 0.426757 Train acc: 0.043333
Epoch: 0/1 Iteration: 5 Validation loss: 0.324521 Validation acc: 0.042333
Epoch: 0/1 Iteration: 6 Train loss: 0.361942 Train acc: 0.037000
Epoch: 0/1 Iteration: 7 Train loss: 0.303807 Train acc: 0.051000
Epoch: 0/1 Iteration: 8 Train loss: 0.253372 Train acc: 0.042000
Epoch: 0/1 Iteration: 9 Train loss: 0.208799 Train acc: 0.047333
Epoch: 0/1 Iteration: 10 Train loss: 0.174880 Train acc: 0.050333
Epoch: 0/1 Iteration: 10 Validation loss: 0.116342 Validation acc: 0.059333
Epoch: 0/1 Iteration: 11 Train loss: 0.148276 Train acc: 0.042333
Epoch: 0/1 Iteration: 12 Train loss: 0.129374 Tra

Epoch: 0/1 Iteration: 100 Train loss: 0.080938 Train acc: 0.061000
Epoch: 0/1 Iteration: 100 Validation loss: 0.074927 Validation acc: 0.076333
Epoch: 0/1 Iteration: 101 Train loss: 0.080415 Train acc: 0.059333
Epoch: 0/1 Iteration: 102 Train loss: 0.081284 Train acc: 0.052667
Epoch: 0/1 Iteration: 103 Train loss: 0.080635 Train acc: 0.060333
Epoch: 0/1 Iteration: 104 Train loss: 0.080530 Train acc: 0.063667
Epoch: 0/1 Iteration: 105 Train loss: 0.080687 Train acc: 0.067667
Epoch: 0/1 Iteration: 105 Validation loss: 0.075199 Validation acc: 0.075667
Epoch: 0/1 Iteration: 106 Train loss: 0.081131 Train acc: 0.056667
Epoch: 0/1 Iteration: 107 Train loss: 0.080887 Train acc: 0.060000
Epoch: 0/1 Iteration: 108 Train loss: 0.080191 Train acc: 0.069333
Epoch: 0/1 Iteration: 109 Train loss: 0.079674 Train acc: 0.073000
Epoch: 0/1 Iteration: 110 Train loss: 0.080563 Train acc: 0.067000
Epoch: 0/1 Iteration: 110 Validation loss: 0.075050 Validation acc: 0.080333
Epoch: 0/1 Iteration: 111 Train 

Epoch: 0/1 Iteration: 200 Train loss: 0.078792 Train acc: 0.066333
Epoch: 0/1 Iteration: 200 Validation loss: 0.074358 Validation acc: 0.082333
Epoch: 0/1 Iteration: 201 Train loss: 0.078937 Train acc: 0.066667
Epoch: 0/1 Iteration: 202 Train loss: 0.079311 Train acc: 0.070333
Epoch: 0/1 Iteration: 203 Train loss: 0.079211 Train acc: 0.071667
Epoch: 0/1 Iteration: 204 Train loss: 0.079277 Train acc: 0.070333
Epoch: 0/1 Iteration: 205 Train loss: 0.079504 Train acc: 0.063667
Epoch: 0/1 Iteration: 205 Validation loss: 0.074586 Validation acc: 0.080333
Epoch: 0/1 Iteration: 206 Train loss: 0.079636 Train acc: 0.067000
Epoch: 0/1 Iteration: 207 Train loss: 0.079767 Train acc: 0.065333
Epoch: 0/1 Iteration: 208 Train loss: 0.079228 Train acc: 0.072000
Epoch: 0/1 Iteration: 209 Train loss: 0.079632 Train acc: 0.059667
Epoch: 0/1 Iteration: 210 Train loss: 0.079039 Train acc: 0.075333
Epoch: 0/1 Iteration: 210 Validation loss: 0.074193 Validation acc: 0.087000
Epoch: 0/1 Iteration: 211 Train 

Epoch: 0/1 Iteration: 300 Train loss: 0.078383 Train acc: 0.072000
Epoch: 0/1 Iteration: 300 Validation loss: 0.074358 Validation acc: 0.078000
Epoch: 0/1 Iteration: 301 Train loss: 0.077769 Train acc: 0.077333
Epoch: 0/1 Iteration: 302 Train loss: 0.078220 Train acc: 0.075667
Epoch: 0/1 Iteration: 303 Train loss: 0.077911 Train acc: 0.074000
Epoch: 0/1 Iteration: 304 Train loss: 0.078518 Train acc: 0.064667
Epoch: 0/1 Iteration: 305 Train loss: 0.078596 Train acc: 0.068000
Epoch: 0/1 Iteration: 305 Validation loss: 0.074432 Validation acc: 0.079000
Epoch: 0/1 Iteration: 306 Train loss: 0.078167 Train acc: 0.071000
Epoch: 0/1 Iteration: 307 Train loss: 0.078343 Train acc: 0.067000
Epoch: 0/1 Iteration: 308 Train loss: 0.078250 Train acc: 0.065667
Epoch: 0/1 Iteration: 309 Train loss: 0.078254 Train acc: 0.065000
Epoch: 0/1 Iteration: 310 Train loss: 0.077937 Train acc: 0.079667
Epoch: 0/1 Iteration: 310 Validation loss: 0.073981 Validation acc: 0.083333
Epoch: 0/1 Iteration: 311 Train 

Epoch: 0/1 Iteration: 400 Train loss: 0.077697 Train acc: 0.071667
Epoch: 0/1 Iteration: 400 Validation loss: 0.074102 Validation acc: 0.078333
Epoch: 0/1 Iteration: 401 Train loss: 0.076732 Train acc: 0.075667
Epoch: 0/1 Iteration: 402 Train loss: 0.077046 Train acc: 0.079333
Epoch: 0/1 Iteration: 403 Train loss: 0.077752 Train acc: 0.068333
Epoch: 0/1 Iteration: 404 Train loss: 0.077689 Train acc: 0.067667
Epoch: 0/1 Iteration: 405 Train loss: 0.077179 Train acc: 0.072000
Epoch: 0/1 Iteration: 405 Validation loss: 0.073967 Validation acc: 0.081333
Epoch: 0/1 Iteration: 406 Train loss: 0.076663 Train acc: 0.068000
Epoch: 0/1 Iteration: 407 Train loss: 0.077292 Train acc: 0.068000
Epoch: 0/1 Iteration: 408 Train loss: 0.077288 Train acc: 0.066000
Epoch: 0/1 Iteration: 409 Train loss: 0.077250 Train acc: 0.068667
INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.CancelledError'>, Session has been closed.


KeyboardInterrupt: 